In [21]:
import json
import pickle
import datetime

In [62]:
# Assuming that the input from the Web is sent as a dict format
# a function that preprocess the input
def preprocess_nnlp(project):
    # turn the input into adequate variables
    project['launched_year'] = project.apply(lambda x: x['launched'].split(" ")[0].split("-")[0], axis = 1)
    project['launched_month'] = project.apply(lambda x: x['launched'].split(" ")[0].split("-")[1], axis = 1)
    project['launched_day'] = project.apply(lambda x: x['launched'].split(" ")[0].split("-")[2], axis = 1)
    project['launched_date'] = project.apply(lambda x: x['launched'].split(" ")[0].split("-")[1] + x['launched'].split(" ")[0].split("-")[2], axis = 1)
    project['deadline_dt'] = project.apply(lambda x: datetime.datetime.strptime(x['deadline'], "%Y-%m-%d").date(), axis = 1)
    project['launched_dt'] = project.apply(lambda x: datetime.datetime.strptime(x['launched'].split(" ")[0], "%Y-%m-%d").date(), axis = 1)
    project['term'] = project.apply(lambda x: int(str(x['deadline_dt'] - x['launched_dt']).split(" ")[0]), axis = 1)
    project['term_str'] = project.apply(lambda x: str(x['term']), axis = 1)
    
    train_data = project[['main_category', 'currency', 'country', 'usd_goal_real', 'state_bin', 'launched_year', 'launched_month', 'term', 'blurb']]
    
    # Assigning a categorical type to categorical varibles and vectorizing the categories for simplification
    with open('cat_labels.json') as json_file:
        data = json.load(json_file)
    
    train['main_category'] = train_data['main_category'].map(data['main_category'])
    train['currency'] = train_data['currency'].map(data['currency'])
    train['country'] = train_data['country'].map(data['country'])
    
    # get the remaing variables
    train[['usd_goal_real', 'launched_year', 'launched_month', 'term', 'state', 'blurb']] = train_data[['usd_goal_real', 'launched_year', 'launched_month', 'term', 'state_bin', 'blurb']]
    
    # turn categories into integer
    train['category'] = train['category'].astype('int16')
    train['main_category'] = train['main_category'].astype('int8')
    train['currency'] = train['currency'].astype('int8')
    train['country'] = train['country'].astype('int8')
    
    # turning into right variables
    train['launched_year'] = (train['launched_year']).astype('int')
    train['launched_month'] = (train['launched_month']).astype('int')
    
    return train

In [32]:
def preprocess_nlp(project):
    state_id_df = peta[['state', 'state_id']].drop_duplicates().sort_values('state_id')
    state_to_id = dict(state_id_df.values)
    id_to_state = dict(state_id_df[['state_id', 'state']].values)
    
    # import vectorizer
    vectorizer = pickle.load(open('vectorizer.sav', 'rb'))
    
    return vectorizer.transform(project['blurb'])

# The function that retuns the probability

In [37]:
# Assuming that the input from the Web is sent as a dict format
def model(iw):
    # load model
    cb_clf = pickle.load(open('Non_nlp_model', 'rb'))
    lg_clf = pickle.load(open('nlp_model.sav', 'rb'))
    
    # predict
    nnlp_pred = cb_clf.predict(preprocess_nnlp(iw))
    nlp_pred = lg_clf.predict(preprocess_nlp(iw))
    
    # final probability
    final_pred = (nnlp_pred + nlp_pred)[:,1] / 2
    
    return final_pred

# for Testing

In [44]:
# Connecting to the SQL server
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd

In [45]:
%run dbconn.ipynb

In [46]:
conn.autocommit = True

In [47]:
data = pd.DataFrame()

In [57]:
# get state from DB and append to data DF
state = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select * from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    sex = dict(i)
    break

In [60]:
df = pd.DataFrame([sex], columns=sex.keys())
df

,id,collected_date,updated_date,name,blurb,state,category,country,funding_rate,pledged,goal,currency_type,usd_pledged_real,usd_goal_real,launched,deadline,term,term_bin,usd_goal_real_bin,backers
0,1,2019-11-29,2019-11-29,Vintage Reborn: New Vintage Chicken Nugget Pla...,Remastering Jerrys Nuggets rare predecessor - ...,success,Games,,345,34523,10000,USD (*),34523.0,10000.0,2019-10-08,2019-11-15,38,5,5,514.0
